In [ ]:
import sys
sys.path.append("../yolov4/")

In [ ]:
from tool.utils import *
from tool.torch_utils import *
from tool.darknet2pytorch import Darknet
import argparse
from PIL import Image
import cv2

"""hyper parameters"""
use_cuda = False
img_size=256
conf_thres=0.8
nms_thres=0.4

namesfile = '../yolov4/cfg/coco.names'
class_names = load_class_names(namesfile)

def detect_cv2(cfgfile, weightfile, imgfile):
    
    m = Darknet(cfgfile)

    #m.print_network()
    m.load_weights(weightfile)

    if use_cuda:
        m.cuda()

    num_classes = m.num_classes
    

    #img = cv2.imread(imgfile)
    img = imgfile
    sized = cv2.resize(img, (m.width, m.height))
    sized = cv2.cvtColor(sized, cv2.COLOR_BGR2RGB)
    

    for i in range(2):
        start = time.time()
        boxes = do_detect(m, sized, 0.4, 0.6, use_cuda, img_size)
        finish = time.time()

    
    return boxes[0]


In [ ]:
videopath = '../../data/artifacts/videos/terraza.mp4'

%pylab inline 
import cv2
from IPython.display import clear_output

cmap = plt.get_cmap('tab20b')
colors = [cmap(i)[:3] for i in np.linspace(0, 1, 20)]

vid = cv2.VideoCapture(videopath)
#mot_tracker = Sort() 

skip_frames=20
while(True):
#for ii in range(10):
    ret, frame = vid.read()
    if skip_frames>0:
        skip_frames-=1
    else:
        skip_frames=20
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #frame = frame[250:650, 480:1650]
        pilimg = Image.fromarray(frame)
        #detections = detect_image(pilimg)
        boxes = detect_cv2('../yolov4/cfg/yolov4.cfg', '../yolov4/cfg/yolov4.weights', frame)
        
        
        
        img = np.copy(frame)
        colors = np.array([[1, 0, 1], [0, 0, 1], [0, 1, 1], [0, 1, 0], [1, 1, 0], [1, 0, 0]], dtype=np.float32)

        def get_color(c, x, max_val):
            ratio = float(x) / max_val * 5
            i = int(math.floor(ratio))
            j = int(math.ceil(ratio))
            ratio = ratio - i
            r = (1 - ratio) * colors[i][c] + ratio * colors[j][c]
            return int(r * 255)

        width = img.shape[1]
        height = img.shape[0]
        
        
        color = (0, 255, 0) 
        
        clients=0
        employees=0
            
        for i in range(len(boxes)):
            box = boxes[i]
            x1 = int((box[0] - box[2] / 2.0) * width)
            y1 = int((box[1] - box[3] / 2.0) * height)
            x2 = int((box[0] + box[2] / 2.0) * width)
            y2 = int((box[1] + box[3] / 2.0) * height)

            

            if color:
                rgb = color
            else:
                rgb = (255, 0, 0)
                
            if len(box) >= 7 and class_names:
                cls_conf = box[5]
                cls_id = box[6]
                classes = len(class_names)
                offset = cls_id * 123457 % classes
                red = get_color(2, offset, classes)
                green = get_color(1, offset, classes)
                blue = get_color(0, offset, classes)
                if color is None:
                    rgb = (red, green, blue)
                cv2.putText(frame, class_names[cls_id], (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1.2, rgb, 1)

                cv2.rectangle(frame, (x1, y1), (x2, y2), rgb, 1)




        fig=figure(figsize=(24, 16))
        title("Video Stream")
        imshow(frame)
        show()
        clear_output(wait=True)